In [1]:
import os
from types import SimpleNamespace

import dotenv
import openai
import pymongo
import pycmarkgfm
import numpy as np
from IPython import display
from ipywidgets import HTML
from bson import ObjectId

In [2]:
def format_text(text):
    return HTML(pycmarkgfm.markdown_to_html(text))

In [3]:
def connect_to_db(collection):
    dotenv.load_dotenv()
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["ML2-Abschlussprojekt"]
    return db[collection]

In [4]:
def serialize_obj(obj):
    """Recursively convert ObjectId to str and prepare for JSON serialization."""
    if isinstance(obj, ObjectId):
        return str(obj)
    if isinstance(obj, list):
        return [serialize_obj(item) for item in obj]
    if isinstance(obj, dict):
        return {key: serialize_obj(value) for key, value in obj.items()}
    return obj

In [5]:
def get_semester(id):
    collection = connect_to_db("Semester_Data")
    results = collection.find_one({"_id":ObjectId(id)})
    return serialize_obj(results)

In [6]:
def get_pupil(id):
    collection = connect_to_db("Base_Data")
    results = collection.find_one({"_id":ObjectId(id)})
    return serialize_obj(results)

In [7]:
semester = get_semester("665d8ff86ce65f51c7e17e34")
student = get_pupil(semester["student_id"])

In [8]:
student

{'_id': '665d8ff66ce65f51c7e17e2a',
 'firstname': 'Bruce',
 'lastname': 'Banner',
 'gender': 'männlich',
 'birthday': '2009-02-02',
 'school_entry': '2022-08-01',
 'diagnose': 'Schizophrenie',
 'compensation': 'Separater Raum bei Prüfungen',
 'class_teacher': 'Nick Fury',
 'special_education_teacher': 'Miss Marvel'}

In [9]:
semester

{'_id': '665d8ff86ce65f51c7e17e34',
 'student_id': '665d8ff66ce65f51c7e17e2a',
 'semester_name': 'HS23',
 'niveau': 'A',
 'last_progress_talk': 'FS23',
 'last_report': 'FS23',
 'final_assessment': {'allgemeines_lernen': {'AktivTeilnehmen': {'assessment': 'gut',
    'notes': ''},
   'LeistungZeigen': {'assessment': 'sehr gut', 'notes': ''},
   'AufmerksamSein': {'assessment': 'teilweise', 'notes': ''},
   'SchulinhalteMerken': {'assessment': 'nicht immer', 'notes': ''},
   'SchulinhalteAbrufen': {'assessment': 'noch nicht',
    'notes': 'asdsdfadfasd'}}},
 'text': None}

In [10]:
criteria = { 
    "AktivTeilnehmen": 'aktiv am Unterricht teilnehmen',
    "LeistungZeigen": 'im Unterricht Leistung zeigen',
    "AufmerksamSein": 'während dem Unterricht Aufmerksam sein und den Instruktionen der Lehrperson folgen',
    "SchulinhalteMerken": 'sich schulische Inhalte merken',
    "SchulinhalteAbrufen": 'schulische Inhalte bei Bedarf abrufen',
}

In [11]:
assessment = { 
        "AktivTeilnehmen": 'sehr gut',
        "AktivTeilnehmenNotizen": '',
        "LeistungZeigen": 'gut',
        "LeistungZeigenNotizen": 'Bruce hat sich seit dem letzten Semester sehr verbessert',
        "AufmerksamSein": 'nicht immer',
        "AufmerksamSeinNotizen": '',
        "SchulinhalteMerken": 'teilweise',
        "SchulinhalteMerkenNotizen": '',
        "SchulinhalteAbrufen": 'noch nicht',
        "SchulinhalteAbrufenNotizen": '',
}

In [12]:
assessment_namespace = {
    k: SimpleNamespace(ass=assessment[k],notes=assessment[k+"Notizen"])
    for k in criteria
}

In [14]:
name = student["firstname"]
gender = student["gender"]

if gender == "männlich":
    alias_name = "John Doe"
else:
    alias_name = "Jane Doe"

assessment_prompt = {
    criteria.get(k,k):v
    for k,v in assessment.items()
}

assessment_prompt = "\n".join(
    f" - {name} kann {v.ass} {criteria[k]}"+(f" ({v.notes})" if v.notes else "")
    for k,v in assessment_namespace.items()
)
assessment_prompt = assessment_prompt.replace(name, alias_name)

In [16]:
system_content = f"""
Du wirst eine Evaluation eines Schülers oder einer Schülerin bekommen.
Die Evaluation wird in kurzen einfachen Sätzen sein.
Dein Task ist es, diese Sätze zu einem Fliesstext zusammenzubringen.
Der Fliesstext soll professionell sein.
Der Fliesstext soll nur Informationen enthalten, die von der Evaluation mitgegeben werden.
Der Fliesstext soll die Schweizer Rechtschreibung nutzen, das heisst, Scharf-S wird durch Doppel-S ersetzt.
"""

In [13]:
first_prompt = f"""
Bitte mache aus der folgenden Einschätzung einen professionellen Fliesstext:
{assessment_prompt}
"""

In [15]:
messages = [
    {
        "role": "system",
        "content": system_content
    },
    {
        "role": "user",
        "content": """
John Doe kann gut aktiv am Unterricht teilnehmen.
John Doe kann gut im schulischen Umfeld Leistung zeigen.
John Doe kann nicht immer während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
John Doe kann teilweise sich schulische Inhalte merken.
John Doe kann noch nicht schulische Inhalte bei Bedarf abrufen.
        """
    },
    {
        "role": "assistant",
        "content": """
John Doe gelingt es gut, aktiv am Unterricht teilzunehmen.
Ausserdem zeigt er meist gute Leistungen im schulischen Umfeld.
Er kann jedoch während des Unterrichts nicht immer Aufmerksam sein und den Instruktionen der Lehrperson folgen.
Ausserdem hat John Doe teilweise Mühe, sich schulische Inhalte zu merken.
Auch gelingt es ihm noch nicht, die gemerkten Inhalte bei Bedarf abzurufen.
        """
    },
    {
        "role": "user",
        "content": """
Jane Doe kann teilweise aktiv am Unterricht teilnehmen.
Jane Doe kann nicht immer im schulischen Umfeld Leistung zeigen.
Jane Doe kann gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
Jane Doe kann gut sich schulische Inhalte merken.
Jane Doe kann teilweise schulische Inhalte bei Bedarf abrufen.
        """
    },
    {
        "role": "assistant",
        "content": """
Jane Doe kann nur teilweise aktiv am Unterricht teilnehmen.
Nicht immer gelingt es ihr im schulischen Umfeld Leistungen zu zeigen.
Sie kann jedoch gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
Ausserdem merkt sie sich schulische Inhalte gut, kann diese jedoch nur teilweise bei Bedarf abrufen.
        """
    },
    {
        "role": "user",
        "content": first_prompt
    },
]

In [16]:
def get_api_key():
    dotenv.load_dotenv()
    os.getenv("OPENAI_APIKEY")
    return os.getenv("OPENAI_APIKEY")
openai.api_key = get_api_key()

In [17]:
def _openai(messages) -> openai.types.chat.chat_completion.Choice:
    """
    Request is sent to OpenAI
    Return the response in a ChatCompletion-Object
    """
    model_result = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        top_p=1,
    )
    response = model_result.choices[0]
    chain = messages + [response.message]
    return response.message.content, chain

In [18]:
answer, chain = _openai(messages)

In [19]:
format_text(answer)

HTML(value='<p>John Doe zeigt eine sehr gute Fähigkeit, aktiv am Unterricht teilzunehmen.\nZudem kann er gut i…

In [20]:
def follow_up_prompt(prompt, messages):
    messages.append({"role":"user","content":prompt})
    return messages

In [140]:
import textwrap
ex_fmt = "\n".join(
    "> "+
    "\n> ".join(textwrap.wrap(ex.strip()))
    + "\n"
    for ex in text_exmaples
)

text_exmaples = [
"""
Jane Doe zeigt auch bei Fächern, welche sie nicht so interessieren, ansprechende Leistungen.
Es gelingt ihr, aufmerksam zu sein und den Instruktionen der Lehrperson zu folgen.
Dennoch braucht sie meist nochmals eine 1:1 Erklärung, damit sie das Gesagte verarbeiten kann.
Jane Doe hat manchmal Schwierigkeiten, sich Schulinhalte zu merken.
Wenn sie sich jedoch für ein Thema interessiert, kann sie dieses aufnehmen und später wieder abrufen. 
""",
"""
John Doe hat noch Schwierigkeiten, im Unterricht aktiv mitzumachen.
Besonders bei Frontalunterricht bekundet John Doe Mühe, da es ihm sehr schnell zu laut wird.
Dann verweigert er komplett. 
Im normalen Unterrichtsgeschehen gelingt es ihm meist, aufmerksam zu sein und den Instruktionen der Lehrperson zu folgen.
John Doe hat manchmal Schwierigkeiten, sich Schulinhalte zu merken.
Wenn er sich jedoch für ein Thema interessiert, kann er dieses aufnehmen und später wieder abrufen. 
"""
]

In [ ]:
first_prompt = f"""
Bitte mache aus der folgenden Einschätzung einen professionellen Fliesstext:

{assessment_prompt}

Der Text darf trocken formuliert sein.
Im Text soll die Schweizer Rechtschreibung ohne Scharf-S verwendet werden.

Wichtig ist, dass nur Informationen verwendet werden, die im
Assessment stehen und keine weiteren Informationen erfunden werden.
Es sollen keine Empfehlungen für die Zukunft gemacht werden.

Ausserdem sollen passenden Absätze gemacht werden.
Bitte nutze die korrekten Pronomen im Text.
"""